NOTE: redoo p-value when all samples are made. P-values before that are not real but would prob be close enough for plotting and debugging

In [130]:
from pg import DB
db = DB()

In [140]:
sample_average_score_sizes = db.query("Select Distinct size from random_samples_average_score").getresult()
sample_average_score_sizes

[(120,),
 (106,),
 (129,),
 (7,),
 (41,),
 (72,),
 (2,),
 (29,),
 (71,),
 (102,),
 (6,),
 (104,),
 (74,),
 (38,),
 (128,),
 (130,),
 (118,),
 (88,),
 (84,),
 (68,),
 (55,),
 (64,),
 (24,),
 (9,),
 (63,),
 (116,),
 (113,),
 (100,),
 (133,),
 (98,),
 (78,),
 (59,),
 (18,),
 (134,),
 (90,),
 (42,),
 (79,),
 (10,),
 (12,),
 (32,),
 (81,),
 (34,),
 (85,),
 (37,),
 (69,),
 (76,),
 (52,),
 (65,),
 (35,),
 (31,),
 (51,),
 (109,),
 (57,),
 (108,),
 (19,),
 (33,),
 (50,),
 (121,),
 (89,),
 (66,),
 (17,),
 (107,),
 (14,),
 (87,),
 (67,),
 (96,),
 (61,),
 (3,),
 (131,),
 (112,),
 (132,),
 (39,),
 (135,),
 (126,),
 (11,),
 (43,),
 (114,),
 (5,),
 (124,),
 (75,),
 (97,),
 (21,),
 (105,),
 (93,),
 (60,),
 (27,),
 (45,),
 (70,),
 (91,),
 (22,),
 (49,),
 (13,),
 (86,),
 (136,),
 (111,),
 (117,),
 (137,),
 (26,),
 (58,),
 (25,),
 (82,),
 (44,),
 (20,),
 (101,),
 (127,),
 (23,),
 (92,),
 (123,),
 (62,),
 (53,),
 (56,),
 (40,),
 (122,),
 (95,),
 (73,),
 (119,),
 (30,),
 (4,),
 (77,),
 (125,),
 (15,),
 (94

In [138]:
scores = db.query("Select average_score from random_samples_average_score where size= '87'").getresult()
scores

[(0.0191894734954,),
 (0.0320208587831,),
 (0.0209792413512,),
 (0.0413694095903,),
 (0.0321846512187,),
 (0.0388805533642,),
 (0.0262993077276,),
 (0.0397196352812,),
 (0.0351052171959,),
 (0.0398817182783,),
 (0.0250257219762,),
 (0.0248957495878,),
 (0.0242848316552,),
 (0.0270109516599,),
 (0.0241490664781,),
 (0.0291314239437,),
 (0.021266935661,),
 (0.0301805654193,),
 (0.0362946049104,),
 (0.0313978719797,),
 (0.0159146359825,),
 (0.0401487952955,),
 (0.0329601326835,),
 (0.0283077385246,),
 (0.0185433577486,),
 (0.0402513680779,),
 (0.0306697569535,),
 (0.0328395023993,),
 (0.0325600844428,),
 (0.0210137576796,),
 (0.0328929954249,),
 (0.0321127461888,),
 (0.0290737970462,),
 (0.0269380248489,),
 (0.031802250528,),
 (0.0231196262053,),
 (0.0335096903812,),
 (0.0369668656761,),
 (0.0343856800493,),
 (0.031423228047,),
 (0.0208903809734,),
 (0.0301094127516,),
 (0.0260460293307,),
 (0.0194505701382,),
 (0.0199894711747,),
 (0.0324169230865,),
 (0.0207254511942,),
 (0.026058570035

In [50]:
motifs = db.query("SELECT motif_string, average_score, proteins_length, shuffled_average_score, shuffled_proteins_length FROM motif where shuffled_subgraph NOTNULL and subgraph NOTNULL").getresult()


In [38]:
score = motifs[0][1]
size = motifs[0][2]
score

0.022089848033521

In [44]:
total = db.query("Select count(average_score) from random_samples_average_score where size= '87'").getresult()
total[0][0]

10002

In [43]:
larger = db.query("Select count(average_score) from random_samples_average_score where size= '87' and average_score > 0.022089848033521").getresult()
larger[0][0]


7955

In [47]:
larger[0][0]/total[0][0]

0.7953409318136373

In [148]:

for size_t in sample_average_score_sizes:
    size = size_t[0]
    motifs_of_size = db.query("SELECT motif_string, average_score, tt_average_score, proteins_length FROM motifss where average_score NOTNULL and proteins_length = '" + str(size) + "'").getresult()
    motifs_of_size_shuffled = db.query("SELECT motif_string, shuffled_average_score, tt_shuffled_average_score, shuffled_proteins_length FROM motifss where shuffled_average_score NOTNULL and shuffled_proteins_length = '" + str(size) + "'").getresult()
    for motif_t in motifs_of_size:
        if(db.query("SELECT p_value FROM motifss where motif_string = '" + motif_t[0] +"' and p_value NOTNULL").ntuples() > 0):
            continue
        size = motif_t[3]
        average_score = motif_t[1]
        tt_average_score = motif_t[2]
        total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(size) +"'").getresult()[0][0]
        right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(size) + "' and average_score > '" +str(average_score) +"'").getresult()[0][0]
        tt_right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(size) + "' and tt_average_score > '" +str(tt_average_score) +"'").getresult()[0][0]

        p_value = right_of_distribution/total_distribution
        tt_p_value = tt_right_of_distribution/total_distribution        
#         print("real", size, average_score ,motif_t[0], p_value)
        do_not_print = db.query("UPDATE motifss SET p_value = " + str(p_value) + ",tt_p_value = " + str(tt_p_value) + "  WHERE motif_string  = '" + motif_t[0] + "'")

    for motif_shuffled_t in motifs_of_size_shuffled:
        if(db.query("SELECT shuffled_p_value FROM motifss where motif_string = '" + motif_shuffled_t[0] +"' and shuffled_p_value NOTNULL").ntuples() > 0):
            continue
        shuffled_size = motif_shuffled_t[3]
        shuffled_average_score = motif_shuffled_t[1]
        tt_shuffled_average_score = motif_shuffled_t[2]
        total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(shuffled_size) +"'").getresult()[0][0]
        right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(shuffled_size) + "' and average_score > '" +str(shuffled_average_score) +"'").getresult()[0][0]
        tt_right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(shuffled_size) + "' and tt_average_score > '" +str(tt_shuffled_average_score) +"'").getresult()[0][0]
        shuffled_p_value = right_of_distribution/total_distribution
        tt_shuffled_p_value = tt_right_of_distribution/total_distribution

#         print("shuff", shuffled_size, shuffled_average_score,motif_shuffled_t[0], shuffled_p_value)
        do_not_print = db.query("UPDATE motifss SET shuffled_p_value = '" + str(shuffled_p_value) + "', tt_shuffled_p_value = '" + str(tt_shuffled_p_value) + "'WHERE motif_string  = '" + motif_shuffled_t[0] + "'")





KeyboardInterrupt: 

In [88]:
for motif_t in motifs_of_size:
    size = motif_t[2]
    average_score = motif_t[1]
    total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(size) +"'").getresult()[0][0]
    right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(size) + "' and average_score > '" +str(average_score) +"'").getresult()[0][0]
    p_value = right_of_distribution/total_distribution
    print(motif_t[0], p_value)



aaagaacg 0.40491901619676063
gtccaata 0.22425514897020596
gtcccata 0.8434313137372526
aacgtgca 0.027494501099780043
gtgtaacc 0.3327334533093381
ttcgtgtg 0.9607078584283143
taaaacgg 0.5537892421515697
taaaatcg 0.4917016596680664
tacgtttg 0.6809638072385523
tactagcc 0.3935212957408518
taggcaac 0.17456508698260348
taggtcca 0.22585482903419316
tagtctac 0.643871225754849
tatagggg 0.20995800839832032
tatcaccc 0.8996200759848031
tatggacc 0.8953209358128374
tatttcgt 0.8170365926814637
tcaatagc 0.8747250549890022
tcacgctg 0.6720655868826235
tcacgtga 0.13907218556288742
tcacgtgt 0.07428514297140572
tccaatag 0.6162767446510697


KeyboardInterrupt: 

In [89]:
for motif_shuffled_t in motifs_of_size_shuffled:
    shuffled_size = motif_shuffled_t[2]
    shuffled_average_score = motif_shuffled_t[1]
    total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(shuffled_size) +"'").getresult()[0][0]
    right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(shuffled_size) + "' and average_score > '" +str(shuffled_average_score) +"'").getresult()[0][0]
    shuffled_p_value = right_of_distribution/total_distribution
    print(motif_shuffled_t[0], shuffled_p_value)




gtcagccc 1.0
aacgcctg 1.0
gtcgggac 0.863627274545091
tacacggc 0.8674265146970606
taccgagc 0.9992001599680064
taccgggc 0.5404919016196761
tagccaga 0.05508898220355929
tagcgaac 0.3473305338932214
tagcgccc 1.0
taggccac 0.6884623075384922
tcagcgac 1.0
tcacggtg 0.9275144971005799
tcaccagg 0.9994001199760048
tcacgagg 1.0
tcagcccg 0.7258548290341932


KeyboardInterrupt: 

In [90]:
motif_shuffled_t

('tccagcgg', -0.00156009710769118, 34)

In [146]:
# db.query("ALTER TABLE motifss ADD COLUMN p_value float;")
# db.query("ALTER TABLE motifss ADD COLUMN shuffled_p_value float;")
# db.query("ALTER TABLE motifss ADD COLUMN tt_p_value float;")
# db.query("ALTER TABLE motifss ADD COLUMN tt_shuffled_p_value float;")

In [118]:
total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '37'").getresult()[0][0]
right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '37' and average_score > '0.0447684706226442'").getresult()[0][0]


In [119]:
total_distribution

10002

In [120]:
right_of_distribution


603

In [121]:
right_of_distribution/total_distribution



0.0602879424115177